# Google.com Loading Times Dashboard

Interactive dashboard to visualize and analyze loading times for Google.com with 1-5 concurrent users.

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interactive, HBox, VBox, Layout
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

In [ ]:
# Load and prepare data
df = pd.read_csv('loading_times.csv', names=['Timestamp', 'Users', 'Avg_Loading_Time'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Date'] = df['Timestamp'].dt.date
df['Hour'] = df['Timestamp'].dt.hour
df['DayOfWeek'] = df['Timestamp'].dt.day_name()
df['Month'] = df['Timestamp'].dt.month_name()
df['Year'] = df['Timestamp'].dt.year

## Interactive Controls

In [ ]:
# Create interactive widgets
user_filter = widgets.SelectMultiple(
    options=[1, 2, 3, 4, 5],
    value=[1, 2, 3, 4, 5],
    description='Users:',
    disabled=False,
    layout=Layout(width='200px', height='120px')
)

view_type = widgets.Dropdown(
    options=['Time Series', 'By Hour', 'By Day of Week', 'By Month', 'Distribution'],
    value='Time Series',
    description='View:',
    disabled=False,
    layout=Layout(width='300px')
)

show_stats = widgets.Checkbox(
    value=True,
    description='Show Statistics',
    disabled=False,
    indent=False
)

In [ ]:
# Function to update visualizations
def update_dashboard(users, view, show_stats_flag):
    # Filter data
    filtered_df = df[df['Users'].isin(users)]
    
    if len(filtered_df) == 0:
        print("No data available for selected filters.")
        return
    
    # Create output
    output = widgets.Output()
    
    with output:
        clear_output(wait=True)
        
        # Show statistics if enabled
        if show_stats_flag:
            print("="*80)
            print("STATISTICS SUMMARY")
            print("="*80)
            print(f"Total measurements: {len(filtered_df):,}")
            print(f"Date range: {filtered_df['Timestamp'].min().date()} to {filtered_df['Timestamp'].max().date()}")
            print(f"Overall mean: {filtered_df['Avg_Loading_Time'].mean():.2f} ms")
            print(f"Overall median: {filtered_df['Avg_Loading_Time'].median():.2f} ms")
            print(f"Overall std dev: {filtered_df['Avg_Loading_Time'].std():.2f} ms")
            print("\nBy Number of Users:")
            for user in sorted(users):
                user_data = filtered_df[filtered_df['Users'] == user]['Avg_Loading_Time']
                if len(user_data) > 0:
                    print(f"  {user} user(s): Mean = {user_data.mean():.2f} ms, Median = {user_data.median():.2f} ms, Count = {len(user_data)}")
            print("="*80)
            print()
        
        # Create visualizations based on view type
        if view == 'Time Series':
            fig, ax = plt.subplots(figsize=(16, 8))
            for user in sorted(users):
                user_data = filtered_df[filtered_df['Users'] == user].sort_values('Timestamp')
                ax.plot(user_data['Timestamp'], user_data['Avg_Loading_Time'], 
                       label=f'{user} User(s)', marker='o', markersize=4, alpha=0.7)
            ax.set_title('Loading Times Over Time', fontsize=16, fontweight='bold')
            ax.set_xlabel('Date/Time', fontsize=12)
            ax.set_ylabel('Loading Time (ms)', fontsize=12)
            ax.legend(fontsize=10)
            ax.grid(alpha=0.3)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
            
        elif view == 'By Hour':
            fig, axes = plt.subplots(2, 1, figsize=(16, 10))
            
            # Average by hour
            avg_by_hour = filtered_df.groupby('Hour')['Avg_Loading_Time'].mean()
            avg_by_hour.plot(kind='line', ax=axes[0], marker='o', color='darkgreen', linewidth=2)
            axes[0].set_title('Mean Loading Time by Hour of Day (UTC)', fontsize=14, fontweight='bold')
            axes[0].set_xlabel('Hour of Day (0-23)')
            axes[0].set_ylabel('Mean Loading Time (ms)')
            axes[0].grid(alpha=0.3)
            axes[0].set_xticks(range(0, 24))
            
            # Heatmap by hour and users
            pivot_hour_users = filtered_df.pivot_table(values='Avg_Loading_Time', index='Hour', columns='Users', aggfunc='mean')
            sns.heatmap(pivot_hour_users, annot=True, fmt='.0f', cmap='YlOrRd', ax=axes[1], 
                       cbar_kws={'label': 'Loading Time (ms)'})
            axes[1].set_title('Loading Time Heatmap: Hour vs Users', fontsize=14, fontweight='bold')
            axes[1].set_xlabel('Number of Users')
            axes[1].set_ylabel('Hour of Day (UTC)')
            
            plt.tight_layout()
            plt.show()
            
        elif view == 'By Day of Week':
            fig, ax = plt.subplots(figsize=(14, 6))
            day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            avg_by_day = filtered_df.groupby('DayOfWeek')['Avg_Loading_Time'].mean()
            avg_by_day = avg_by_day.reindex([day for day in day_order if day in avg_by_day.index])
            avg_by_day.plot(kind='bar', ax=ax, color='coral')
            ax.set_title('Mean Loading Time by Day of Week', fontsize=14, fontweight='bold')
            ax.set_xlabel('Day of Week')
            ax.set_ylabel('Mean Loading Time (ms)')
            ax.tick_params(axis='x', rotation=45)
            ax.grid(axis='y', alpha=0.3)
            plt.tight_layout()
            plt.show()
            
        elif view == 'By Month':
            fig, ax = plt.subplots(figsize=(14, 6))
            avg_by_month = filtered_df.groupby('Month')['Avg_Loading_Time'].mean()
            avg_by_month.plot(kind='bar', ax=ax, color='teal')
            ax.set_title('Mean Loading Time by Month', fontsize=14, fontweight='bold')
            ax.set_xlabel('Month')
            ax.set_ylabel('Mean Loading Time (ms)')
            ax.tick_params(axis='x', rotation=45)
            ax.grid(axis='y', alpha=0.3)
            plt.tight_layout()
            plt.show()
            
        elif view == 'Distribution':
            fig, axes = plt.subplots(1, 2, figsize=(16, 6))
            
            # Box plot
            filtered_df.boxplot(column='Avg_Loading_Time', by='Users', ax=axes[0])
            axes[0].set_title('Loading Time Distribution by Users', fontsize=14, fontweight='bold')
            axes[0].set_xlabel('Number of Users')
            axes[0].set_ylabel('Loading Time (ms)')
            axes[0].get_figure().suptitle('')
            
            # Violin plot
            sns.violinplot(data=filtered_df, x='Users', y='Avg_Loading_Time', ax=axes[1], palette='Set2')
            axes[1].set_title('Loading Time Distribution (Violin Plot)', fontsize=14, fontweight='bold')
            axes[1].set_xlabel('Number of Users')
            axes[1].set_ylabel('Loading Time (ms)')
            axes[1].grid(axis='y', alpha=0.3)
            
            plt.tight_layout()
            plt.show()
    
    display(output)

## Dashboard

In [ ]:
# Create interactive dashboard
dashboard = interactive(update_dashboard, 
                       users=user_filter, 
                       view=view_type,
                       show_stats_flag=show_stats)

# Layout
controls = VBox([user_filter, view_type, show_stats], 
                layout=Layout(padding='10px'))

display(VBox([controls, dashboard.children[-1]]))